In [51]:
!pip install streamlit joblib torch scikit-learn matplotlib




In [52]:
!pip install streamlit pyngrok -q

In [53]:
!ls -lh


total 8.2M
-rw-r--r-- 1 root root 219K Oct 30 21:58 lstm_model.pth
-rw-r--r-- 1 root root 5.2K Oct 30 22:40 predictwise_results.csv
drwxr-xr-x 1 root root 4.0K Oct 29 13:38 sample_data
-rw-r--r-- 1 root root 2.3K Oct 30 22:03 streamlit_app.py
-rw-r--r-- 1 root root 7.9M Oct 30 21:58 xgb_model.pkl


In [54]:
!pkill streamlit || echo "No existing Streamlit process"
!pkill ngrok || echo "No existing ngrok process"


No existing Streamlit process
No existing ngrok process


In [55]:
!ngrok config add-authtoken 34o4tTpQlwAaqZem3SexUPFUpUh_4kfcYm7FHM4LVMBBApJbV


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [56]:
from pyngrok import ngrok

port = 8501  # You can change to 8502 if you like
public_url = ngrok.connect(port)
print(" Streamlit App URL:", public_url)


 Streamlit App URL: NgrokTunnel: "https://cristobal-subpectoral-rubie.ngrok-free.dev" -> "http://localhost:8501"


In [57]:
import streamlit as st
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import joblib
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score


#  PAGE CONFIGURATION

st.set_page_config(page_title="PredictWise – Sales Forecast Dashboard", layout="wide", page_icon="📈")

st.markdown(
    """
    <style>
        body { background-color: #0e1117; color: white; }
        .main-title { text-align: center; color: #00BFFF; font-size: 36px; font-weight: bold; }
        .subtitle { text-align: center; color: #BBBBBB; font-size: 18px; margin-bottom: 20px; }
        .metric-box {
            background-color: #1e2130;
            padding: 15px;
            border-radius: 10px;
            text-align: center;
            color: white;
            margin-bottom: 10px;
        }
        .stDownloadButton button {
            background-color: #00BFFF;
            color: white;
            border-radius: 8px;
        }
    </style>
    """,
    unsafe_allow_html=True
)

st.markdown("<p class='main-title'>📈 PredictWise – LSTM vs XGBoost Sales Forecast</p>", unsafe_allow_html=True)
st.markdown("<p class='subtitle'>Trained models compared on validation data below. Upload your own file to test predictions.</p>", unsafe_allow_html=True)


# LSTM MODEL DEFINITION

class LSTMModel(nn.Module):
    def __init__(self, input_size=18, hidden_size=64, num_layers=2, output_size=1):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out


# LOAD MODELS

@st.cache_resource
def load_models():
    xgb_model = joblib.load("xgb_model.pkl")

    lstm_model = LSTMModel(input_size=18, hidden_size=64, num_layers=2, output_size=1)
    state_dict = torch.load("lstm_model.pth", map_location=torch.device("cpu"))
    lstm_model.load_state_dict(state_dict)
    lstm_model.eval()
    return xgb_model, lstm_model

xgb_model, lstm_model = load_models()


# LOAD TRAINED RESULTS (predictwise_results.csv)

try:
    df_results = pd.read_csv("predictwise_results.csv")
    st.success("✅ Model results loaded successfully.")
except FileNotFoundError:
    st.warning("⚠️ Could not find 'predictwise_results.csv'. Showing random sample data.")
    df_results = pd.DataFrame({
        "Actual_Sales": np.random.randint(2000, 9000, 50),
        "Pred_LSTM": np.random.randint(2500, 9500, 50),
        "Pred_XGBoost": np.random.randint(2200, 9700, 50)
    })


# SHOW TRAINED MODEL RESULTS

st.markdown("### 🧠 Model Performance on Validation Data")

# Compute metrics
rmse_xgb = np.sqrt(mean_squared_error(df_results["Actual_Sales"], df_results["Pred_XGBoost"]))
rmse_lstm = np.sqrt(mean_squared_error(df_results["Actual_Sales"], df_results["Pred_LSTM"]))
r2_xgb = r2_score(df_results["Actual_Sales"], df_results["Pred_XGBoost"])
r2_lstm = r2_score(df_results["Actual_Sales"], df_results["Pred_LSTM"])

# Layout for metrics
col1, col2 = st.columns(2)
with col1:
    st.markdown("#### ⚙️ XGBoost Performance")
    st.markdown(f"<div class='metric-box'><b>RMSE:</b> {rmse_xgb:.2f}<br><b>R² Score:</b> {r2_xgb:.4f}</div>", unsafe_allow_html=True)
with col2:
    st.markdown("#### ⚙️ LSTM Performance")
    st.markdown(f"<div class='metric-box'><b>RMSE:</b> {rmse_lstm:.2f}<br><b>R² Score:</b> {r2_lstm:.4f}</div>", unsafe_allow_html=True)

# Visualization – side by side plots
col3, col4 = st.columns(2)
with col3:
    st.markdown("#### 📈 XGBoost vs Actual Sales")
    fig1, ax1 = plt.subplots(figsize=(6, 4))
    ax1.plot(df_results["Actual_Sales"][:80], label="Actual", color="white")
    ax1.plot(df_results["Pred_XGBoost"][:80], label="XGBoost", color="#00FF7F", linestyle="--")
    ax1.legend()
    st.pyplot(fig1)

with col4:
    st.markdown("#### 📈 LSTM vs Actual Sales")
    fig2, ax2 = plt.subplots(figsize=(6, 4))
    ax2.plot(df_results["Actual_Sales"][:80], label="Actual", color="white")
    ax2.plot(df_results["Pred_LSTM"][:80], label="LSTM", color="#FFA500", linestyle="--")
    ax2.legend()
    st.pyplot(fig2)


# SIDEBAR: USER UPLOAD

st.sidebar.title("📂 Upload CSV for Prediction")
st.sidebar.markdown("Upload your own feature file to get predictions below:")

uploaded_file = st.sidebar.file_uploader("Choose your CSV file", type=["csv"])


# GENERATE PREDICTIONS FOR USER FILE

if uploaded_file:
    user_df = pd.read_csv(uploaded_file)
    st.markdown("### 🔍 Uploaded Data Preview")
    st.dataframe(user_df.head())

    model_choice = st.selectbox("Select model for prediction:", ["XGBoost", "LSTM"])

    if st.button("🚀 Generate Predictions"):
        X = user_df.values.astype(float)
        if model_choice == "XGBoost":
            preds = xgb_model.predict(X)
        else:
            X_tensor = torch.tensor(X, dtype=torch.float32).unsqueeze(1)
            with torch.no_grad():
                preds = lstm_model(X_tensor).cpu().numpy().flatten()

        user_df["Predicted_Sales"] = preds
        st.subheader("📊 Predicted Results")
        st.dataframe(user_df.head())

        fig, ax = plt.subplots(figsize=(8, 4))
        ax.plot(user_df["Predicted_Sales"], label="Predicted Sales", color="#00BFFF")
        ax.legend()
        ax.grid(True, alpha=0.3)
        st.pyplot(fig)

        csv = user_df.to_csv(index=False).encode("utf-8")
        st.download_button("📥 Download Predictions", csv, file_name="Predicted_Sales.csv", mime="text/csv")

else:
    st.sidebar.info("👆 Upload your test CSV to generate new predictions.")




2025-10-30 23:35:30.114 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 23:35:30.116 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 23:35:30.117 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 23:35:30.118 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 23:35:30.120 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 23:35:30.122 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 23:35:30.123 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 23:35:30.124 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [ ]:
!streamlit run streamlit_app.py --server.port 8501 --server.address 0.0.0.0 >/dev/null
